### 1.1 Import library

In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
import keras.backend as K

# from keras_tqdm import TQDMNotebookCallback

### 1.2 Load dataset

In [ ]:
Df = pd.read_excel('../input/priceindia/Prices.xlsx', sheet_name='Daily')

In [ ]:

df = Df.loc[8:, ['Unnamed: 3', 'Unnamed: 4']]

df = df.dropna()

df['Unnamed: 3'] = pd.to_datetime(df['Unnamed: 3'])

df = df.set_index('Unnamed: 3')

data = df.loc[:, ['Unnamed: 4']]

In [ ]:
df.head()

### 2. visualization

In [ ]:
df.plot(figsize=(15, 10),color='r')
plt.ylabel("Gold ETF Prices")
plt.title("Gold ETF Price Series")
plt.show()

In [ ]:
df.describe()

### 3. Pre training

In [ ]:
train, test = np.split(df, [int(.8*len(df))])
train.shape

In [ ]:
sc = MinMaxScaler()

train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)

In [ ]:
X_train = train_sc[:-1]
y_train = train_sc[1:]

X_test = test_sc[:-1]
y_test = test_sc[1:]

In [ ]:
ax = train.plot()
test.plot(ax=ax, figsize=(15,10))
plt.legend(['train', 'test'])

In [ ]:
type(train)

In [ ]:
sc = MinMaxScaler()

train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)

In [ ]:
train_sc.shape

In [ ]:
X_train = X_train[:, None]
X_test = X_test[:, None]

In [ ]:
X_train.shape

### 4. Create Neural network

In [ ]:
K.clear_session()
model = Sequential()

model.add(LSTM(50, activation='relu', input_shape=(1, 1)))

model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

In [ ]:
%%time
history = model.fit(X_train, y_train, epochs=500, verbose=0)

In [ ]:
%%time
y_pred = model.predict(X_test)
plt.figure(figsize=(15, 10))
plt.plot(y_test)
plt.plot(y_pred)
plt.legend(['real', 'predict'])
plt.show()

sc.inverse_transform(y_pred)[-1]

print(history.history)

plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 60])
plt.show()

In [ ]:
loss = history.history['loss']

epochs = range(len(loss)) # Get number of epochs

In [ ]:
plt.plot(epochs, loss, 'r')
plt.title('Training loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss"])

plt.figure(figsize=(15, 10))
plt.show()

### 5. the goodness of the fit

In [ ]:
score = model.evaluate(X_test,y_test, verbose=1)
print(score)

In [ ]:
score = sum(np.absolute(y_pred - y_test)) / sc.scale_
print(score)

In [ ]:
import math
from sklearn.metrics import mean_squared_error

def rmse(y_test, y_pred):
    return math.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)
print("Accuracy: ", 100 - 100 * rmse(y_test, y_pred))

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred) * 100)

In [ ]:
model.save('my_model')

### 7. Optimal

In [ ]:
results = pd.DataFrame()
epochs = [10]
for i in epochs:
    error_scores = list()
    for j in range(30):
        history = model.fit(X_train, y_train, epochs=i, verbose=0)
        y_pred = model.predict(X_test)
        error_scores.append(rmse(y_test, y_pred))
    
    
    results[str(i)] = error_scores

print(results.describe())
# save boxplot
results.boxplot()
plt.show()